### Importing

In [1]:
import pyodbc 

### Connecting to DB

In [2]:
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=MousaPC;'
                      'Database=Vodafone;'
                      'Trusted_Connection=yes;')

cursor = conn.cursor()

#### Asking for ColName, Table names

In [5]:
# function to get input of New Columns
def getinput1():
  colName = input ("Insert new column name: ")
  devTable = input ("Insert development table name: ")
  prodTable = input ("Insert production table name: ")
  keyCol = input ("Key Column to make joins: ")
  srcCount = input ("Insert source count: ")
  filterCol = input ("Insert column name to apply filters on (0 => no filter): ")
  if filterCol != "0":  
      fromValue = input ("From value: ")
      toValue = input ("To value: ")
  else:
    fromValue = "0";
    toValue = "0";
  groupByCol = input ("Insert column name to apply Group by (0 => no grouping): ")
  dataType = input ("Insert the expected data type: (str/int)")
  # filter, n columns?
    #if src count = 0 , use production count
    # if src count != 0, use production count and src count
  
  return colName, devTable, prodTable, keyCol, srcCount, filterCol, fromValue, toValue, groupByCol, dataType

colName, devTable, prodTable, keyCol, srcCount, filterCol,fromValue,toValue, groupByCol, dataType = getinput1()


Insert new column name: Address
Insert development table name: Contract_Dev
Insert production table name: Contract_Prd
Insert source count: 5
Insert column name to apply filters on (0 => no filter): 0
Insert column name to apply Group by (0 => no grouping): s
Insert the expected data type: (str/int)s


In [89]:
#Dev SQL
sqlSelect = f"SELECT COUNT(*) FROM {devTable} "
sqlFilter = " "
sqlGroupBy = " "

if filterCol != "0": #Filter Applied
    sqlFilter = f" WHERE {filterCol} BETWEEN '{fromValue}' AND '{toValue}' "
if groupByCol != "0": #Group By Applied
    sqlGroupBy = f" GROUP BY {groupByCol} "
sqlCommand = sqlSelect + sqlFilter + sqlGroupBy 
sqlCommand += ";"
cursor.execute(sqlCommand);
devCount = cursor.fetchall()
print(f"devCount = {devCount}")
#Prod SQL
sqlSelect = f"SELECT COUNT(*) FROM {prodTable} "
sqlCommand = sqlSelect + sqlFilter + sqlGroupBy
sqlCommand += ";"
cursor.execute(sqlCommand)
prdCount = cursor.fetchall()
print(f"prdCount = {prdCount}")
#Output
if srcCount != "0":
    if srcCount == f"{devCount}":
        print ("Source vs. Dev Tamaaammm")
        print("")
    else: 
        print ("Source vs. Dev not Tamam")

devCount = []
prdCount = []
Source vs. Dev not Tamam


In [ ]:
error = (abs(devCount - prdCount)/prdCount)*100
if error < 1:
    print("Dev vs. Prod Tamam")
else:
    print(f"Error {error}")

#### Validation 3


In [7]:
sqlCommand = f"SELECT * INTO TempTable FROM {devTable}; ALTER TABLE TempTable DROP Column {colName}"
sqlCommand += ";"
cursor.execute(sqlCommand);

#### Prod Minus DEV

In [14]:
sqlCommand =  f" SELECT {prodTable}.* FROM {prodTable} LEFT JOIN TempTable ON ({prodTable}.{keyCol} = TempTable.{keyCol}) WHERE TempTable.MSISDN IS NULL"
cursor.execute(sqlCommand);
rows = cursor.fetchall()
if len(rows) > 0:
    print("Difference Not Zero: ")
    print(rows)
else:
    print("Production Minus DEV Zero!")

Difference Not Zero: 
[('02145215  ', '444       ', '2021-08-07')]


#### DEV Minus Prod

In [16]:
sqlCommand =  f" SELECT TempTable.* FROM TempTable LEFT JOIN {prodTable} ON (TempTable.{keyCol} = {prodTable}.{keyCol}) WHERE {prodTable}.MSISDN IS NULL"
cursor.execute(sqlCommand);
rows = cursor.fetchall()
if len(rows) > 0:
    print("Difference Not Zero: ")
    print(rows)
else:
    print("Dev Minus Prod Zero!")

Difference Not Zero: 
[('0125545   ', '444       ', '2021-08-07'), ('01233541  ', '555       ', '2021-08-06')]
